In [12]:
import json
from datetime import datetime

import requests

# Step 1: Download
url = "https://fhir.org/guides/stats2/registry.json"
response = requests.get(url)
raw = response.json()
packages = raw["packages"]


# Step 2: Extract simplified data
output = []

for key, val in packages.items():
    if "#" not in key:
        continue
    name, version = key.split("#")
    deps = val.get("dependencies", {})
    output.append(
        {
            "id": key,
            "name": name,
            "version": version,
            "dependencies": [f"{dep}#{v}" for dep, v in deps.items()],
        }
    )

output_data = {"created": datetime.now().isoformat(), "packages": output}


# Step 3: Save to slim JSON file
with open("slim_fhir_graph.json", "w") as f:
    json.dump(output_data, f, indent=2)

In [ ]:
packages

{'name': 'hl7.eu.fhir.pcsp',
 'version': '0.2.0',
 'tools-version': 3,
 'type': 'IG',
 'date': '20230905122018',
 'license': 'CC-BY-SA-4.0',
 'canonical': 'http://hl7.eu/fhir/ig/pcsp',
 'notForPublication': True,
 'url': 'https://build.fhir.org/ig/hl7-eu/pcsp/',
 'title': 'PanCareSurPass Project HL7 FHIR Implementation Guide',
 'description': 'PanCareSurPass is an EU project aiming to conduct a robust assessment of the Cancer Survivorship Passport (SurPass). (built Tue, Sep 5, 2023 12:20+0000+00:00)',
 'fhirVersions': ['4.0.1'],
 'dependencies': {'hl7.fhir.r4.core': '4.0.1',
  'hl7.terminology.r4': '5.0.0',
  'hl7.fhir.uv.extensions.r4': '1.0.0',
  'hl7.fhir.uv.ips': 'current',
  'hl7.fhir.us.mcode': 'current'},
 'author': 'PanCareSurPass Project',
 'maintainers': [{'name': 'PanCareSurPass Project',
   'url': 'https://www.pancaresurpass.eu/'},
  {'name': 'PanCareSurPass Project', 'url': 'https://www.pancaresurpass.eu/'}],
 'directories': {'lib': 'package', 'example': 'example'}}

In [16]:
for k, v in raw["packages"].items():
    print(k)

au.digitalhealth.r4#current
hl7.fhir.uv.patient-corrections#current
ihe.iti.mhd#current
ch.fhir.ig.ch-core#current
hl7.fhir.us.pacio-pfe#current
hl7.fhir.uv.emedicinal-product-info#current
kl.fhir.dk.children#current
hl7.fhir.us.qicore#current
hl7.org.nz.fhir.ig.hpi#current
hl7.fhir.uv.v2mappings#current
ch.fhir.ig.ch-emed#current
hl7.fhir.fi.base#current
hl7.fhir.uv.cpg#current
cinc.fhir.ig#current
hl7.fhir.uv.dicom-sr#current
hl7.fhir.us.ph-library#current
medcom.fhir.dk.terminology#current
hl7.fhir.uv.shc-vaccination#current
example.fhir.uv.myig#current
ch.fhir.ig.ch-lab-order#current
hl7.fhir.be.public-health#current
hl7.fhir.fi.smart#current
hl7.fhir.us.vrsandbox#current
ihe.iti.sips#current
hl7.fhir.us.codex-radiation-therapy#current
hl7.fhir.be.mycarenet#current
hl7.fhir.uv.ichom-breast-cancer#current
hl7.fhir.us.central-cancer-registry-reporting#current
hl7.fhir.us.specialty-rx#current
hl7.fhir.uv.vulcan-schedule#current
medcom.fhir.dk.homecareobservation#current
hl7.fhir.uv.vh

In [17]:
import json
from datetime import datetime

import requests

# Step 1: Download
url = "https://fhir.org/guides/stats2/registry.json"
response = requests.get(url)
raw = response.json()
packages = raw["packages"]

# Step 2: Extract simplified data
output = []
existing_ids = set()
dep_ids = set()

for key, val in packages.items():
    if "#" not in key:
        continue
    name, version = key.split("#")
    deps = val.get("dependencies", {})
    dep_list = [f"{dep}#{v}" for dep, v in deps.items()]
    output.append(
        {"id": key, "name": name, "version": version, "dependencies": dep_list}
    )
    existing_ids.add(key)
    dep_ids.update(dep_list)

# Step 3: Add missing dependencies as nodes with no deps
missing_dep_ids = dep_ids - existing_ids
for dep_id in sorted(missing_dep_ids):
    if "#" in dep_id:
        name, version = dep_id.split("#")
        output.append(
            {"id": dep_id, "name": name, "version": version, "dependencies": []}
        )

# Step 4: Add timestamp
output_data = {"created": datetime.now().isoformat(), "packages": output}

# Step 5: Save to file
with open("slim_fhir_graph.json", "w") as f:
    json.dump(output_data, f, indent=2)